#### Clean 311 Data - England, John
---
311 Repository: https://andrew-friedman.github.io/jkan/datasets/

The functions in the notebook below were used to clean the 311 datasets from the following cities:
- Buffalo, NY
- Kansas City, MO
- Austin, TX

#### Buffalo, NY: July 2008 - present

(https://data.buffalony.gov/Quality-of-Life/311-Service-Requests/whkc-e5vr/about_data)

In [ ]:
import pandas as pd

# Define Function to Clean Buffalo 311 Data
def clean_buffalo(input_path='../data/buffalo.csv', output_path='../data/buffalo_cleaned.csv'):
    try:
        # Load Data
        df = pd.read_csv(input_path)

        # Streamline Data Transformation
        # Selecting desired features, renaming, and converting datatypes
        df = df[['Open Date', 'Closed Date', 'Subject']]
        df.rename(columns={'Open Date': 'open_date', 'Closed Date': 'closed_date', 'Subject': 'department'}, inplace=True)
        df['open_date'] = pd.to_datetime(df['open_date'])
        df['closed_date'] = pd.to_datetime(df['closed_date'])

        # Handling Missing Data
        df.dropna(subset=['open_date','closed_date','department'], inplace=True)

        # Add constant column
        df['city'] = 'Buffalo'

        # Save Cleaned Dataframe
        df.to_csv(output_path, index=False)

    except Exception as e:
        print(f"An error occurred: {e}")

# Function Call
clean_buffalo()

#### Baltimore, MD: 2003 - Current

https://data.baltimorecity.gov/City-Services/311-Customer-Service-Requests/9agw-sxsr

In [28]:
import pandas as pd

# Define Function to Clean Buffalo 311 Data
def clean_baltimore(input_path='../data/baltimore.csv', output_path='../data/baltimore_cleaned.csv'):
    try:
        # Load Data
        df = pd.read_csv(input_path)

        # Streamline Data Transformation
        # Selecting desired features, renaming, and converting datatypes
        df = df[['CreatedDate', 'CloseDate', 'Agency']]
        df.rename(columns={'CreatedDate': 'open_date', 'CloseDate': 'closed_date', 'Agency': 'department'}, inplace=True)
        df['open_date'] = pd.to_datetime(df['open_date'])
        df['closed_date'] = pd.to_datetime(df['closed_date'])

        # Handling Missing Data
        df.dropna(subset=['open_date','closed_date','department'], inplace=True)

        # Add constant column
        df['city'] = 'Baltimore'

        # Save Cleaned Dataframe
        df.to_csv(output_path, index=False)
    
    except Exception as e:
        print(f"An error occurred: {e}")

clean_baltimore()

/var/folders/32/h_9nbrl13s73c2_31mnv0cdm0000gn/T/ipykernel_56914/2594871866.py:7: DtypeWarning: Columns (17,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_path)


#### Kansas CIty, MO 311 Call Center Service Requests: 2007 - March 2021



(https://data.kcmo.org/311/311-Call-Center-Service-Requests-2007-March-2021/7at3-sxhp/about_data)

In [ ]:
### Export Server Error when obtaining data.

#### Austin, TX 311 Call Center Requests: 01/03/2014 - Present

https://data.austintexas.gov/Utilities-and-City-Services/Austin-311-Public-Data/xwdj-i9he/about_data

In [ ]:
### Department Column N/A